# 本文件说明

# 基本设置

In [46]:
import jieba
import sys
import os

%matplotlib inline
import numpy as np
import pandas as pd
import pre_cor
import json

import warnings
warnings.filterwarnings('ignore')

In [47]:
def get_txt_encode(file_path):
    '''
    Function:
        获取txt文档的编码格式
    Arguments:
        file_path -> string -- 文档所在路径
    Return:
        encode -> string -- txt文档的编码格式
    '''
    decode_set = ['utf-8','gb18030','ISO-8859-2','gb2312','gbk', 'ANSI'] # 编码集
    encode = 0
    for k in decode_set:
        try :
            file = open(file_path, 'r', encoding = k).read()
            # print('\n\t读取成功： %s \n\t编码方式为： %s' %(file_path, k))
            encode = k
            break
        except :
            continue
    if encode:
        return encode
    else :
        return 'No encode'

In [48]:
def load_raw_files(data_folder, from_row = 4):
    filename_list = os.listdir(data_folder)
    data = []
    for filename in filename_list:
        file_path = os.path.join(data_folder, filename)
        data_a = []
        data_b = []
        data_c = [] # 例外
        try :
            encoding = get_txt_encode(file_path)
            f = open(file_path, "r+", encoding = encoding)
            for index, line in enumerate(f):
                if index > from_row: # 跳过 N 行
                    if line.startswith('A：'):
                        data_a.append(line.replace('A：', ''))
                    elif line.startswith('B：'):
                        data_b.append(line.replace('B：', ''))
                    else :
                        data_c.append(line)

            data.append({'id': filename, 'A':"。".join(data_a),
                         'B':"。".join(data_b), 'C':"。".join(data_c)})                 
            f.close()               
        except Exception as e:
            print('---------------')
            print(e)
            print(file_path)
            continue  
            
    return data

# 导入原始数据

## 诈骗电话记录

In [49]:
data_folder_pos = r'D:\XH\Python_Project\notebook\proj_youlang\data\201180720'
pos_raw_data = load_raw_files(data_folder_pos, from_row = 0)
print(len(pos_raw_data))
pos_raw_data[0]

3


{'A': '嗯啊！喂喂。\n。啊！我我我在你这是实心的一个号码吗？\n。对对对我现在是在室内然后安静的一个环境然后是多少。\n',
 'B': '喂一个。\n。诶\n。那我正好玩那么好。\n。啊！就是我之前那样的号码。\n。哦哦哦。\n。呃，这个是那个车子零跟你们说就是多医生采集是吧！\n。用移动的号码。\n。嗯哦。\n。是你你们那边。',
 'C': '语种：汉语\t性别：男和女\n。主叫：201806280909\t被叫：2018062809010\n。开始时间：2018-06-28 09:08:24\t结束时间：2018-06-28 09:14:01\n。\n',
 'id': 'test - 副本 (2).txt'}

In [50]:
if not os.path.exists('data/'):
    os.mkdir('data')

In [51]:
with open("data/raw_pos.json",'w',encoding='utf-8') as json_file:
        json.dump(pos_raw_data,json_file,ensure_ascii=False) 

## 非诈骗电话记录

In [52]:
data_folder_neg = r'D:\XH\Python_Project\notebook\proj_youlang\data\201180720'
neg_raw_data = load_raw_files(data_folder_neg, from_row = 0)
print(len(neg_raw_data))
neg_raw_data[0]

3


{'A': '嗯啊！喂喂。\n。啊！我我我在你这是实心的一个号码吗？\n。对对对我现在是在室内然后安静的一个环境然后是多少。\n',
 'B': '喂一个。\n。诶\n。那我正好玩那么好。\n。啊！就是我之前那样的号码。\n。哦哦哦。\n。呃，这个是那个车子零跟你们说就是多医生采集是吧！\n。用移动的号码。\n。嗯哦。\n。是你你们那边。',
 'C': '语种：汉语\t性别：男和女\n。主叫：201806280909\t被叫：2018062809010\n。开始时间：2018-06-28 09:08:24\t结束时间：2018-06-28 09:14:01\n。\n',
 'id': 'test - 副本 (2).txt'}

In [53]:
with open("data/raw_neg.json",'w',encoding='utf-8') as json_file:
        json.dump(neg_raw_data,json_file,ensure_ascii=False) 

# 预处理

In [54]:
# 诈骗电话
pos_pre = [pre_cor.handle_content(content['A']) + ':AB: ' + pre_cor.handle_content(content['B']) for content in pos_raw_data]
print(len(pos_pre))
print(pos_pre[0])

save_filename = 'data/pos_pre_20180723.txt'
print(save_filename)
fid = open(save_filename, "w+", encoding='UTF-8')
for data in pos_pre:
    fid.write(data + '\n')
fid.close()

3
喂喂 实心 一个 号码 对 对 对 现在 室内 然后 安静 一个 环境 然后 多少 :AB: 一个 诶 正好 玩 之前 号码 哦哦哦 车子 零 跟 说 多 医生 采集 移动 号码 
data/pos_pre_20180723.txt


In [55]:
# 非诈骗电话
neg_pre = [pre_cor.handle_content(content['A']) + ':AB: ' + pre_cor.handle_content(content['B']) for content in neg_raw_data]
print(len(neg_pre))
print(neg_pre[0])

save_filename = 'data/neg_pre_20180723.txt'
print(save_filename)
fid = open(save_filename, "w+", encoding='UTF-8')
for data in neg_pre:
    fid.write(data + '\n')
fid.close()

3
喂喂 实心 一个 号码 对 对 对 现在 室内 然后 安静 一个 环境 然后 多少 :AB: 一个 诶 正好 玩 之前 号码 哦哦哦 车子 零 跟 说 多 医生 采集 移动 号码 
data/neg_pre_20180723.txt


# 保存文件